![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_AlbertForMultipleChoice.ipynb)

## Import ONNX AlbertForMultipleChoice models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `AlbertForMultipleChoice` is only available since in `Spark NLP 5.6.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import ALBERT models trained/fine-tuned for question answering via `AlbertForMultipleChoice` or `AlbertForMultipleChoice`. These models are usually under `Multiple Choice` category and have `bert` in their labels
- Reference: [AlbertForMultipleChoice](https://huggingface.co/docs/transformers/main/en/model_doc/albert#transformers.AlbertForMultipleChoice)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.

In [1]:
!pip install -q --upgrade transformers[onnx] optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use the treained model above as an example and load it as a `ORTModelForMultipleChoice`, representing an ONNX model.

In [2]:
!pip install -q --upgrade transformers[onnx] optimum

In [4]:
from optimum.onnxruntime import ORTModelForMultipleChoice

MODEL_NAME = "Ariffiq99/CRAB_COPA_KUCI_e_care_albert_Base_Finetuned"
ONNX_MODEL_PATH = f"onnx_models/albert_multiple_choice"

ort_model = ORTModelForMultipleChoice.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(ONNX_MODEL_PATH)

Let's have a look inside these two directories and see what we are dealing with:

In [6]:
!ls -lh {ONNX_MODEL_PATH}

total 48M
-rw-r--r-- 1 root root  871 Dec 27 19:03 config.json
-rw-r--r-- 1 root root  45M Dec 27 19:03 model.onnx
-rw-r--r-- 1 root root  970 Dec 27 19:03 special_tokens_map.json
-rw-r--r-- 1 root root 743K Dec 27 19:03 spiece.model
-rw-r--r-- 1 root root 1.5K Dec 27 19:03 tokenizer_config.json
-rw-r--r-- 1 root root 2.2M Dec 27 19:03 tokenizer.json


We need the `spiece.model` for the Tokenizer. This is the same for every model, these are assets (saved in /assets) needed for tokenization inside Spark NLP.

In [7]:
!mkdir {ONNX_MODEL_PATH}/assets

In [10]:
!mv {ONNX_MODEL_PATH}/spiece.model {ONNX_MODEL_PATH}/assets/

Voila! We have our vocab.txt inside assets directory

In [11]:
!ls -lR {ONNX_MODEL_PATH}

onnx_models/albert_multiple_choice:
total 48312
drwxr-xr-x 2 root root     4096 Dec 27 19:04 assets
-rw-r--r-- 1 root root      871 Dec 27 19:03 config.json
-rw-r--r-- 1 root root 47180962 Dec 27 19:03 model.onnx
-rw-r--r-- 1 root root      970 Dec 27 19:03 special_tokens_map.json
-rw-r--r-- 1 root root     1442 Dec 27 19:03 tokenizer_config.json
-rw-r--r-- 1 root root  2272611 Dec 27 19:03 tokenizer.json

onnx_models/albert_multiple_choice/assets:
total 744
-rw-r--r-- 1 root root 760289 Dec 27 19:03 spiece.model


## Import and Save AlbertForMultipleChoice in Spark NLP

- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
!wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

In [15]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

Apache Spark version: 3.5.3


- Let's use `loadSavedModel` functon in `AlbertForMultipleChoice` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `AlbertForMultipleChoice` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [17]:
from sparknlp.annotator import *
from sparknlp.base import *


albertMultpleChoiceClassifier = AlbertForMultipleChoice.loadSavedModel(
     f"{ONNX_MODEL_PATH}",
     spark
 )\
  .setInputCols(["document_question", "document_context"])\
  .setOutputCol("answer")

Let's save it on disk so it is easier to be moved around and also be used later via .load function

In [18]:
albertMultpleChoiceClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [19]:
!rm -rf {ONNX_MODEL_PATH}

Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny `AlbertForMultipleChoice` model in Spark NLP 🚀 pipeline!

In [20]:
 testing_data = [
            ("In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.",
             "It is eaten with a fork and a knife, It is eaten while held in the hand."),

            ("The Eiffel Tower is located in which country?",
             "Germany, France, Italy"),

            ("Which animal is known as the king of the jungle?",
             "Lion, Elephant, Tiger, Leopard"),

            ("Water boils at what temperature?",
             "90°C, 120°C, 100°C"),

            ("Which planet is known as the Red Planet?",
             "Jupiter, Mars, Venus"),

            ("Which language is primarily spoken in Brazil?",
             "Spanish, Portuguese, English"),

            ("The Great Wall of China was built to protect against invasions from which group?",
             "The Greeks, The Romans, The Mongols, The Persians"),

            ("Which chemical element has the symbol 'O'?",
             "Oxygenm, Osmium, Ozone"),

            ("Which continent is the Sahara Desert located in?",
             "Asia, Africa, South America"),

            ("Which artist painted the Mona Lisa?",
             "Vincent van Gogh, Leonardo da Vinci, Pablo Picasso")
        ]

In [21]:
testing_df = spark.createDataFrame(testing_data, ["question", "choices"])
testing_df.show(truncate=False)

+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|question                                                                                  |choices                                                                 |
+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.|It is eaten with a fork and a knife, It is eaten while held in the hand.|
|The Eiffel Tower is located in which country?                                             |Germany, France, Italy                                                  |
|Which animal is known as the king of the jungle?                                          |Lion, Elephant, Tiger, Leopard                                          |
|Wat

In [23]:
document_assembler = MultiDocumentAssembler() \
            .setInputCols(["question", "choices"]) \
            .setOutputCols(["document_question", "document_choices"])

albert_for_multiple_choice = AlbertForMultipleChoice() \
    .load("./{}_spark_nlp_onnx".format(MODEL_NAME)) \
    .setInputCols(["document_question", "document_choices"])\
    .setOutputCol("answer") \
    .setBatchSize(4)

pipeline = Pipeline(stages=[document_assembler, albert_for_multiple_choice])
pipeline_model = pipeline.fit(testing_df)

pipeline_df = pipeline_model.transform(testing_df)

pipeline_df.select("answer").show(truncate=False)

+------------------------------------------------------------------------------------------------------------+
|answer                                                                                                      |
+------------------------------------------------------------------------------------------------------------+
|[{chunk, 0, 35,  It is eaten while held in the hand., {sentence -> 0, chunk -> 0, score -> 0.55574197}, []}]|
|[{chunk, 0, 5,  Italy, {sentence -> 0, chunk -> 0, score -> 0.3497405}, []}]                                |
|[{chunk, 0, 8,  Elephant, {sentence -> 0, chunk -> 0, score -> 0.28558698}, []}]                            |
|[{chunk, 0, 5,  100°C, {sentence -> 0, chunk -> 0, score -> 0.34499714}, []}]                               |
|[{chunk, 0, 4,  Mars, {sentence -> 0, chunk -> 0, score -> 0.3803456}, []}]                                 |
|[{chunk, 0, 10,  Portuguese, {sentence -> 0, chunk -> 0, score -> 0.36515844}, []}]                         |
|